<a href="https://colab.research.google.com/github/FabioHerrera97/FlavorMiner/blob/main/FlavorMiner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FlavorMiner: Instructions to Run

This script contains the code necessary to run FlavorMiner. In oreder to run this code the user needs to organize the input data in an excel file containing either the PubChem ID or the isomeric smiles of the input compounds.

Name the column containing the isomeric smiles "Isomeric Smiles". This way there is no need to manipulate the code to run the analyzes.

IMPORTANT: This script requires scikit-learn version 1.2.2. The models trained to perform the respective predictions were trained and saved in joblib files with this version, and a different version with end up with and error at the moment to run the code. Use the code below to install it, if necessary

In [ ]:
!pip install -U scikit-learn==1.2.2

In [ ]:
!pip install "numpy<2.0" pandas --upgrade --force-reinstall

# 1. Upload the data

The first step to use FlavorMiner is to upload the excel file containing the data. To do this in Google Colab go to *Files> Upload>Select the file>Open*.

Then click the three dots on the right of the uploaded file and copy the path.

In [ ]:
import pandas as pd

'''URL of the Excel file'''

file_url = 'https://github.com/FabioHerrera97/FlavorMiner/raw/main/Data/Example.xlsx' # Copy here the path of the input file

''' Read the Excel file'''

data = pd.read_excel(file_url)

''' Print the contents of the DataFrame'''

data.head()

# 2. Matching the input data with the flavor database.

IMPORTANT: This match is performed with the isomeric smiles, not the name of the compounds due to the lack of standardization in compounds name.

This part of the code takes the provided isomeric smiles and perform a database match to exclude from the prediction those entries with already reported flavor profiles. This avoids adding unnnecessary uncertainty when there is experimental data available.

In [ ]:
''' Read the Excel file'''

database = pd.read_excel('https://github.com/FabioHerrera97/FlavorMiner/raw/main/Data/LabelDataBase.xlsx')

''' Print the contents of the DataFrame'''

database.head()

In [ ]:
db_matches = data[data['Isomeric Smiles'].isin(database['Isomeric Smiles'])]
db_matches.head()

For this matches the flavor profile is assigned according the report in the database as well as a probability of 100% as their flavor profile is experimentally validated. Additionally, the source of the data recorded in the database is provided.

In [ ]:
columns = ['Bitter', 'Floral', 'Fruity', 'Off_flavor', 'Nutty', 'Sour', 'Sweet', 'Source']

result_dict = {col: [] for col in columns}

for smiles in db_matches['Isomeric Smiles']:
    for col in columns:
        values = database[database['Isomeric Smiles'] == smiles][col].values
        result_dict[col].append(values[0])

for col in columns:
    db_matches = db_matches.copy()
    db_matches[col] = result_dict[col]

db_matches.head()

This data can be stored in an excel file. Run the following cell if you want to store it.

In [ ]:
db_matches.to_excel('Database matches.xlsx', index=False)

# 3. Obtaining the data without reported flavor profile

The next step is obtaining the set of compounds without database matches. The structure of this molecules will be used for the prediction.

In [ ]:
non_matches = data[~data['Isomeric Smiles'].isin(database['Isomeric Smiles'])]
non_matches.head()

# 4. Featurizing the compounds

In order to run the prediction it is required to transform the molecular structure into the mathematical representation that the trained algorithms require: RDKit molecular descriptors and Extended Connectivity Fingerprint.

For this, RDKit python library needs to be installed, as it is not a default library in Google Colab.

In [ ]:
!pip install rdkit-pypi

The first representation obtained are the 200 RDKit molecular descriptors. This descriptors are obtained using the isomeric smiles as input.

In [ ]:
''' Import the RDKit libraries for molecular descriptor calculation'''

from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit import Chem

''' Create a fucntion that extract all the RDKit molecular descriptors'''

def RDkit_descriptors(smiles):
    mols = [Chem.MolFromSmiles(i) for i in smiles]
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()

    Mol_descriptors =[]

    for mol in mols:
        descriptors = calc.CalcDescriptors(mol)
        Mol_descriptors.append(descriptors)
    return Mol_descriptors,desc_names

''' Call the fucntion that extract all the RDKit molecular descriptors'''

Mol_descriptors,desc_names = RDkit_descriptors(non_matches ['Isomeric Smiles'])

df_RDKit_descriptors = pd.DataFrame(Mol_descriptors,columns=desc_names)

df_RDKit_descriptors.head()

Before training the respective algorithms, the descriptors were cleaned by colineality and variance. Therefore, not all the 200 molecular descriptors will be accepted as input by the respective algorithms.

It is necessary to filter the input descriptors. For this, the file containing the test data is used to filter the input columns.

In [ ]:
RDKit_test = pd.read_excel('https://github.com/FabioHerrera97/FlavorMiner/raw/main/Data/RDKit_test.xlsx')

x_RDKit_data = RDKit_test.drop(['Bitter', 'Floral', 'Fruity', 'Off_flavor', 'Nutty', 'Sour', 'Sweet'], axis=1)

input_columns_names_RDKit = x_RDKit_data.columns.tolist()
input_columns_names_RDKit

input_RDKit = df_RDKit_descriptors[input_columns_names_RDKit]

The first representation obtained are the Extended Connectivity Fingerprint, with a radio of 2. This results in a binary vector with a lenth of 1024 bits.  

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit import Chem
from rdkit.Chem import AllChem

''' Generate a molecule from each isomeric smiles'''

smiles_non_matches = non_matches ['Isomeric Smiles']
mols = [Chem.MolFromSmiles(i) for i in smiles_non_matches]

''' Configure the extended connectivity fingerprint'''

radius=2
nBits=1024

''' Calculate the ECFP for each molecule'''

ECFP2 = [AllChem.GetMorganFingerprintAsBitVect(mol,radius=radius, nBits=nBits) for mol in mols]

ecfp2_name = [f'Bit_{i}' for i in range(nBits)]
ecfp2_bits = [list(l) for l in ECFP2]
df_ecfp_2 = pd.DataFrame(ecfp2_bits, index = non_matches.index, columns = ecfp2_name)
df_ecfp_2.head()

Before training the respective algorithms, the descriptors were cleaned by colineality and variance. Therefore, not all the 200 molecular descriptors will be accepted as input by the respective algorithms.

It is necessary to filter the input descriptors. For this, the file containing the test data is used to filter the input columns.

In [ ]:
columns_ecfp = pd.read_excel('https://github.com/FabioHerrera97/FlavorMiner/raw/main/Data/ECFP_test.xlsx')

x_ECFP_data = columns_ecfp.drop(['Bitter', 'Floral', 'Fruity', 'Off_flavor', 'Nutty', 'Sour', 'Sweet'], axis=1)

input_bits = x_ECFP_data.columns.tolist()

input_ECFP = df_ecfp_2[input_bits]

input_ECFP.head()

# 5. Performing the prediction on those compounds with unknown flavor profiles.

The first step to perform the prediction is to import the models. These models are stored in a Zenodo repository ([https://zenodo.org/records/8435106](https://)). These model were stored in Zeno because they exceded the maximum size allowed by GitHub.

After importing the models, the prediction is run. Similarly, the probability is calculated as measure of the confidence of the prediction for each entry.

This process is first performed for the models trained with Extended Connectivity Fingerprint.

In [ ]:
import joblib
import numpy as np

ecfp_model_urls = [
    'https://zenodo.org/records/8435106/files/Bitter_Random_Forest_SMOTE.sav?download=1',
    'https://zenodo.org/records/8435106/files/Sour_KNN_SMOTE.sav?download=1',
    'https://zenodo.org/records/8435106/files/Fruity_Random_Forest_SMOTE.sav?download=1',
    'https://zenodo.org/records/10033243/files/Sweet_updated_RF_Final.sav?download=1',
]

# Download the model files from Zenodo and load them
ecfp_models = []

for model_url in ecfp_model_urls:
    model_file_name = model_url.split('/')[-1]
    !wget -O {model_file_name} {model_url}
    model = joblib.load(model_file_name)
    ecfp_models.append(model)

predictions_ecfp = []
positive_probabilities_ecfp = []

for model in ecfp_models:
    probabilities = model.predict_proba(input_ECFP)
    positive_probabilities = probabilities[:, 0]
    predictions = model.predict(input_ECFP)
    predictions_ecfp.append(predictions)
    positive_probabilities_ecfp.append(positive_probabilities)

Bitter, Sour, Fruity, Sweet = predictions_ecfp
Bitter_pos_proba, Sour_pos_proba, Fruity_pos_proba, Sweet_pos_proba = positive_probabilities_ecfp


Then, the process is repeated for the models trained with RDKit molecular descriptors.

In [ ]:
# Define the Zenodo model file URLs

rdkit_model_urls = [
    'https://zenodo.org/records/8435106/files/Off_flavor_Random_Forest_SMOTE_RDKit.sav?download=1',
    'https://zenodo.org/records/8435106/files/Floral_Random_Forest_SMOTE_RDKit.sav?download=1',
    'https://zenodo.org/records/8435106/files/Nutty_Random_Forest_SMOTE_RDKit.sav?download=1'
]

# Download the model files from Zenodo and load them

rdkit_models = []
positive_probabilities_rdkit = []
negative_probabilities_rdkit = []
predictions_rdkit = []

for model_url in rdkit_model_urls:
    model_file_name = model_url.split('/')[-1]
    !wget -O {model_file_name} {model_url}
    model = joblib.load(model_file_name)
    rdkit_models.append(model)

for model in rdkit_models:
    probabilities = model.predict_proba(input_RDKit)
    positive_probabilities = probabilities[:, 1]
    negative_probabilities = probabilities[:, 0]
    predictions = model.predict(input_RDKit)
    predictions_rdkit.append(predictions)
    positive_probabilities_rdkit.append(positive_probabilities)
    negative_probabilities_rdkit.append(negative_probabilities)

Off_flavor, Floral, Nutty = predictions_rdkit
Off_flavor_pos_proba, Floral_pos_proba, Nutty_pos_proba = positive_probabilities_rdkit
Off_flavor_neg_proba, Floral_neg_proba, Nutty_neg_proba = negative_probabilities_rdkit

# 6. Results saving and visualization

The results of the predictions are then organized in a table. This table contains the preexisting information in the input data,the flavor profile of the compounds and the source of this flavor profile (wether it was a database match or a prediction)

First, the predicted labels as well as a column indicating that this data is the result of a prediction are added to the data with uknown flavor profiles.


In [ ]:
Source = ["prediction"] * (len(non_matches))

In [ ]:
# Create a dictionary with the lists as values
new_columns = {
    'Bitter': Bitter,
    'Floral': Floral,
    'Fruity': Fruity,
    'Off_flavor': Off_flavor,
    'Nutty': Nutty,
    'Sour': Sour,
    'Sweet': Sweet,
    'Source': Source
}

new_non_matches = non_matches.copy()  # Create a copy to avoid modifying the original DataFrame

for column_name, values in new_columns.items():
    new_non_matches[column_name] = values

new_non_matches.head()

Second, the database matches are combined with predictions and plot using radar curve. This radar curve contains the frequency of compounds containing in their flavor profiles the 7 target flavors.

In [ ]:
combined_data = pd.concat([db_matches, new_non_matches], ignore_index=True)

combined_data.sample(5)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

categories = ['Bitter', 'Floral', 'Fruity', 'Nutty', 'Sweet', 'Off_flavor', 'Sour']
values = [combined_data[col].sum() for col in categories]

num_categories = len(categories)
angles = np.linspace(0, 2 * np.pi, num_categories, endpoint=False).tolist()
angles += angles[:1]

values += values[:1]

plt.figure(figsize=(8, 6))
ax = plt.subplot(111, polar=True)
ax.fill(angles, values, 'b', alpha=0.25)
ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)

plt.savefig('Flavor profile.jpeg', format='jpeg', dpi=2000)

Third, the data is stored in an excel file. Even though this data is combined, the dataframe contains a column name Source indicating if the result id the product of a prediction or a database hit.

Additionally, the results of the 7 binary variables associated with the flavor are combined into a single variable, and these binary variables are dropped.

In [ ]:
new_combined_data = combined_data.copy()

columns=['Bitter', 'Floral', 'Fruity', 'Off_flavor', 'Nutty', 'Sour', 'Sweet']

flavor_profiles = []

for col in columns:
    flavor_profiles.append(col)

new_combined_data['Flavor_profile'] = new_combined_data[flavor_profiles].apply(lambda x: ', '.join(x.index[x == 1]), axis=1)

new_combined_data = new_combined_data.drop(columns=flavor_profiles)

new_combined_data.to_excel('Flavor profile.xlsx', index=False)

Finally, a new dataframe containing only data resulted from the prediction is created. This dataframe contains a column with the probability calculated during the prediction. This probability is offered a measure of the confidence of the prediction.

This final probability is calculated as the average probability of the flavor notes predicted as positive for each entry. The results are sorted in descending oreder and stored in an excel file

In [ ]:
new_columns_2 = {
    'Bitter': Bitter,
    'Floral': Floral,
    'Fruity': Fruity,
    'Off_flavor': Off_flavor,
    'Nutty': Nutty,
    'Sour': Sour,
    'Sweet': Sweet,

    'Bitter probability': Bitter_pos_proba,
    'Sour probability': Sour_pos_proba,
    'Fruity probability': Fruity_pos_proba,
    'Sweet probability': Sweet_pos_proba,
    'Off_flavor probability': Off_flavor_pos_proba,
    'Floral probability': Floral_pos_proba,
    'Nutty probability': Nutty_pos_proba,
}

probability_data = non_matches.copy()  # Create a copy to avoid modifying the original DataFrame

for column_name, values in new_columns_2.items():
    probability_data [column_name] = values

probability_data .head()

In [ ]:
columns_to_check = ['Bitter', 'Sour', 'Fruity', 'Sweet', 'Off_flavor', 'Floral', 'Nutty']

# Create a mask for entries with value 1 in any of the flavor columns
flavor_mask = probability_data[columns_to_check].eq(1).any(axis=1)

# Create a 'profile' column containing names of columns set to 1
probability_data['Profile'] = probability_data[columns_to_check].apply(
    lambda row: ', '.join([col for col, val in zip(columns_to_check, row) if val == 1]), axis=1)

# Calculate average positive and negative probabilities for entries with value 1 in any flavor column
probability_data['Average Positive Probability'] = probability_data.loc[flavor_mask][[f"{col} probability" for col in columns_to_check]].mean(axis=1)

In [ ]:
probability_data.to_excel('Probability data.xlsx', index=False)